In [1]:
from pathlib import Path

import torch
import torch.nn.functional as F
from sklearn.metrics import f1_score
from torch_geometric.nn import GCNConv
from torch_geometric.loader import DataLoader
from torch_geometric.transforms import NormalizeFeatures

from CarcassonneGNN.data.dataset import CarcassonneDataset

In [2]:
dataset = CarcassonneDataset(root=Path('../../data'))

print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object.

print()
print(data)
print('===========================================================================================================')

# Gather some statistics about the graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')

Dataset: CarcassonneDataset(78522):
Number of graphs: 78522
Number of features: 8
Number of classes: 2

Data(edge_index=[2, 1310], tile=[486], tileRotation=[486], candidate=[486], Link.Internal=[1310], Link.Tile=[1310], Link.Feature=[1310], x=[486, 8], edge_attributes=[1310, 3], y=[486], pos=[486, 2])
Number of nodes: 486
Number of edges: 1310
Average node degree: 2.70


## Train/Test/Val Split

In [3]:
train_frac = 0.7
test_frac = 0.15
val_frac = 0.15

dataset = dataset.shuffle()

train_dataset = dataset[:int(len(dataset)*train_frac)]
test_dataset = dataset[len(train_dataset):len(train_dataset) + int(len(dataset)*test_frac)]
val_dataset = dataset[len(train_dataset)+len(test_dataset):]

print(f'Train: {len(train_dataset)} | Test: {len(test_dataset)} | Val: {len(val_dataset)}')

Train: 54965 | Test: 11778 | Val: 11779


## Loader

In [4]:
loader = DataLoader(train_dataset, batch_size=32)

## Model

In [5]:
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        # self.conv2 = GCNConv(16, dataset.num_classes)
        self.conv2 = GCNConv(16, 1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = x.float()

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return x #F.log_softmax(x, dim=1)



## Train

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(1):
    running_loss = 0.

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in enumerate(train_dataset):

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        out = model(data)

        # Compute the loss and its gradients
        # loss = F.nll_loss(out, data.y.long(), weight=torch.Tensor([data.y.mean(), 10.0]))
        candidate_mask = data.x[:,3].bool().logical_not()
        n_nodes = int((data.candidate[candidate_mask] == data.candidate[candidate_mask][0]).sum())

        pred = out[candidate_mask].reshape([-1,n_nodes]).mean(axis=1)
        target = data.y[candidate_mask].reshape([-1,n_nodes]).mean(axis=1).float()

        loss = F.mse_loss(pred, target) # https://github.com/pyg-team/pytorch_geometric/issues/3794
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        if i % 10000 == 9999:
            last_loss = running_loss / 100 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            running_loss = 0.

  batch 10000 loss: 4.76251526635202
  batch 20000 loss: 4.877079368676708
  batch 30000 loss: 4.866090806048915
  batch 40000 loss: 4.744409085348844
  batch 50000 loss: 4.725807165609294


## Eval

In [10]:
model.eval()
f1s = []
for i, data in enumerate(test_dataset):
#     pred = model(data).argmax(dim=1)
    data = test_dataset[0]

    pred = model(data).argmax(dim=1)

    f1 = f1_score(data.y, pred)
    f1s.append(f1)

In [11]:
f1s

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0

In [13]:
model.eval()

data = test_dataset[0]

pred = model(data)

pred.squeeze(), data.y.float().squeeze(), pred.squeeze().argmax(), data.y.float().squeeze().argmax()

(tensor([0.0600, 0.0624, 0.0624, 0.0646, 0.0477, 0.0527, 0.0490, 0.0480, 0.0627,
         0.0571, 0.0581, 0.0548, 0.0472, 0.0513, 0.0480, 0.0523, 0.0509, 0.0524,
         0.0506, 0.0463, 0.0491, 0.0489, 0.0477, 0.0478, 0.0478, 0.0526, 0.0538,
         0.0518, 0.0526, 0.0519, 0.0516, 0.0525, 0.0618, 0.0526, 0.0477, 0.0590,
         0.0474, 0.0476, 0.0465, 0.0466, 0.0463, 0.0465, 0.0463, 0.0463, 0.0463,
         0.0463, 0.0463, 0.0463, 0.0463, 0.0463, 0.0463, 0.0463, 0.0463, 0.0463,
         0.0463, 0.0463, 0.0463, 0.0463, 0.0469, 0.0573, 0.0550, 0.0468, 0.0463,
         0.0463, 0.0463, 0.0463, 0.0463, 0.0463, 0.0486, 0.0512, 0.0545, 0.0563,
         0.0563, 0.0572, 0.0684, 0.0707, 0.0705, 0.0735, 0.0463, 0.0463, 0.0463,
         0.0463, 0.0468, 0.0474, 0.0476, 0.0474, 0.0552, 0.0592, 0.0597, 0.0592,
         0.0502, 0.0507, 0.0499, 0.0512, 0.0626, 0.0636, 0.0623, 0.0657, 0.0748,
         0.0774, 0.0746, 0.0800, 0.0463, 0.0463, 0.0463, 0.0463, 0.0769, 0.0792,
         0.0767, 0.0827, 0.0

In [19]:
pred.squeeze()[data.y.bool()]

tensor([0.0626, 0.0636, 0.0623, 0.0657, 0.0623, 0.0657, 0.0626, 0.0636],
       grad_fn=<IndexBackward0>)

In [15]:
data.y.float()

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1.,
        1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.])

In [22]:
p = pred.squeeze()[data.candidate != 0]


tensor([0.0545, 0.0563, 0.0563, 0.0572, 0.0684, 0.0707, 0.0705, 0.0735, 0.0463,
        0.0463, 0.0463, 0.0463, 0.0468, 0.0474, 0.0476, 0.0474, 0.0552, 0.0592,
        0.0597, 0.0592, 0.0502, 0.0507, 0.0499, 0.0512, 0.0626, 0.0636, 0.0623,
        0.0657, 0.0748, 0.0774, 0.0746, 0.0800, 0.0463, 0.0463, 0.0463, 0.0463,
        0.0769, 0.0792, 0.0767, 0.0827, 0.0463, 0.0463, 0.0463, 0.0463, 0.0631,
        0.0648, 0.0634, 0.0670, 0.0563, 0.0572, 0.0545, 0.0563, 0.0705, 0.0735,
        0.0684, 0.0707, 0.0463, 0.0463, 0.0463, 0.0463, 0.0476, 0.0474, 0.0468,
        0.0474, 0.0597, 0.0592, 0.0552, 0.0592, 0.0499, 0.0512, 0.0502, 0.0507,
        0.0623, 0.0657, 0.0626, 0.0636, 0.0746, 0.0800, 0.0748, 0.0774, 0.0463,
        0.0463, 0.0463, 0.0463, 0.0767, 0.0827, 0.0769, 0.0792, 0.0463, 0.0463,
        0.0463, 0.0463, 0.0634, 0.0670, 0.0631, 0.0648],
       grad_fn=<IndexBackward0>)

In [23]:
data.candidate

tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,
         1,  1,  2,  2,  2,  2,  3,  3,  3,  3,  4,  4,  4,  4,  5,  5,  5,  5,
         6,  6,  6,  6,  7,  7,  7,  7,  8,  8,  8,  8,  9,  9,  9,  9, 10, 10,
        10, 10, 11, 11, 11, 11, 12, 12, 12, 12, 13, 13, 13, 13, 14, 14, 14, 14,
        15, 15, 15, 15, 16, 16, 16, 16, 17, 17, 17, 17, 18, 18, 18, 18, 19, 19,
        19, 19, 20, 20, 20, 20, 21, 21, 21, 21, 22, 22, 22, 22, 23, 23, 23, 23,
        24, 24, 24, 24])

In [40]:
candidate_avg_mask = torch.concat([(data.candidate == i).unsqueeze(dim=1) for i in range(data.candidate.max())], axis=1)

In [52]:
candidate_avg_mask.multiply(pred).sum(axis=0), candidate_avg_mask.multiply(pred).sum(axis=0)[7], candidate_avg_mask.multiply(pred).sum(axis=0)[19]

(tensor([3.5129, 0.2242, 0.2831, 0.1852, 0.1893, 0.2334, 0.2020, 0.2542, 0.3067,
         0.1852, 0.3155, 0.1852, 0.2583, 0.2242, 0.2831, 0.1852, 0.1893, 0.2334,
         0.2020, 0.2542, 0.3067, 0.1852, 0.3155, 0.1852],
        grad_fn=<SumBackward1>),
 tensor(0.2542, grad_fn=<SelectBackward0>),
 tensor(0.2542, grad_fn=<SelectBackward0>))

In [45]:
data.candidate[data.y.bool()]

tensor([ 7,  7,  7,  7, 19, 19, 19, 19])

In [53]:
candidate_avg_mask.multiply(data.y.unsqueeze(dim=1).float()).sum(axis=0)

tensor([0., 0., 0., 0., 0., 0., 0., 4., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 4., 0., 0., 0., 0.])